In [1]:
import collections
import logging
import os
import pathlib
import re
import string
import sys
import time

import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow as tf

In [2]:
from bpemb import BPEmb

In [3]:
bpemb_en=BPEmb(lang="en", dim=200, vs=100000)
bpemb_pt=BPEmb(lang="pt", dim=200, vs=100000)

C:\Users\mohit\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [4]:
bpemb_pt.encode("Stratford")

['▁stratford']

In [5]:
bpemb_pt.encode("Oi como você está")

['▁oi', '▁como', '▁você', '▁está']

In [6]:
ids1=bpemb_en.encode_ids("vostalanj")

In [7]:
ids2=bpemb_en.encode_ids("Soccer")

In [8]:
ids1


[97847, 8510, 99943]

In [9]:
ids2

[6528]

In [27]:
examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True,
                               as_supervised=True)
train_examples, val_examples = examples['train'], examples['validation']
type(train_examples)

tensorflow.python.data.ops.dataset_ops._OptionsDataset

In [11]:
for pt_examples, en_examples in train_examples.batch(3).take(1):
  for pt in pt_examples.numpy():
    print(pt.decode('utf-8'))

  print()

  for en in en_examples.numpy():
    print(en.decode('utf-8'))
    
print()

agora aqui temos imagens sendo extraídas em tempo real diretamente do feed ,
os astrónomos acreditam que cada estrela da galáxia tem um planeta , e especulam que até um quinto deles tem um planeta do tipo da terra que poderá ter vida , mas ainda não vimos nenhum deles .
agora : um , dois , três , vai .

now here are live images being pulled straight from the feed .
astronomers now believe that every star in the galaxy has a planet , and they speculate that up to one fifth of them have an earth-like planet that might be able to harbor life , but we have n't seen any of them .
so : one , two , three , go .



In [12]:
list_of_strings = [tf.compat.as_str_any(en_examples.numpy()) for tensor in en_examples] 

In [13]:
encoded = bpemb_en.encode(list_of_strings)

In [14]:
encoded

[['▁',
  '[',
  'b',
  "'",
  'now',
  '▁here',
  '▁are',
  '▁live',
  '▁images',
  '▁being',
  '▁pulled',
  '▁straight',
  '▁from',
  '▁the',
  '▁feed',
  '▁.',
  "'",
  '\n',
  '▁b',
  '"',
  'astron',
  'omers',
  '▁now',
  '▁believe',
  '▁that',
  '▁every',
  '▁star',
  '▁in',
  '▁the',
  '▁galaxy',
  '▁has',
  '▁a',
  '▁planet',
  '▁,',
  '▁and',
  '▁they',
  '▁speculate',
  '▁that',
  '▁up',
  '▁to',
  '▁one',
  '▁fifth',
  '▁of',
  '▁them',
  '▁have',
  '▁an',
  '▁earth',
  '-',
  'like',
  '▁planet',
  '▁that',
  '▁might',
  '▁be',
  '▁able',
  '▁to',
  '▁harbor',
  '▁life',
  '▁,',
  '▁but',
  '▁we',
  '▁have',
  '▁n',
  "'",
  't',
  '▁seen',
  '▁any',
  '▁of',
  '▁them',
  '▁."',
  '\n',
  '▁b',
  "'",
  'so',
  '▁:',
  '▁one',
  '▁,',
  '▁two',
  '▁,',
  '▁three',
  '▁,',
  '▁go',
  '▁.',
  "'",
  ']'],
 ['▁',
  '[',
  'b',
  "'",
  'now',
  '▁here',
  '▁are',
  '▁live',
  '▁images',
  '▁being',
  '▁pulled',
  '▁straight',
  '▁from',
  '▁the',
  '▁feed',
  '▁.',
  "'",
  '\

In [15]:
ids3 = bpemb_en.encode_ids(list_of_strings)

In [16]:
random_string = "but what if it were active ?"
ids=bpemb_en.encode(random_string)

In [17]:
ids

['▁but', '▁what', '▁if', '▁it', '▁were', '▁active', '▁?']

In [18]:
#Now we will setup an input pipeline

In [23]:
#we need to make tokenizer pairs for both and hence we will define functions to tokenize the same when we pass one pt string
#and one english string as input

def tokenize_pairs(pt, en):
    list_of_strings1 =  [tf.compat.as_str_any(pt.numpy()) for tensor in pt]
    pt = bpemb_pt.encode_ids(list_of_strings1)
    pt = tf.convert_to_tensor(pt)
    
#to equalise length we will convert them from a ragged vector to a dense vector
    pt = pt.to_tensor()
    
    list_of_strings2 =  [tf.compat.as_str_any(en.numpy()) for tensor in en]
    
    en = bpemb_pt.encode_ids(en)
    en = tf.convert_to_tensor(en)
    en = en.to_tensor()
    
    return pt, en

In [24]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64

In [25]:
tf.executing_eagerly()
@tf.function
def make_batches(dataset):
    dataset = dataset.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).map(tokenize_pairs, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)
    return dataset

In [26]:
train_examples= make_batches(train_examples)
val_examples = make_batches(val_examples)

OperatorNotAllowedInGraphError: in user code:

    <ipython-input-25-1ec0473deae4>:4 make_batches  *
        dataset = dataset.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).map(tokenize_pairs, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)
    <ipython-input-19-218eb703c6e8>:5 tokenize_pairs  *
        list_of_strings1 =  [tf.compat.as_str_any(pt.numpy()) for tensor in pt]
    C:\Users\mohit\anaconda3\lib\site-packages\tensorflow\python\framework\ops.py:520 __iter__
        self._disallow_iteration()
    C:\Users\mohit\anaconda3\lib\site-packages\tensorflow\python\framework\ops.py:513 _disallow_iteration
        self._disallow_when_autograph_enabled("iterating over `tf.Tensor`")
    C:\Users\mohit\anaconda3\lib\site-packages\tensorflow\python\framework\ops.py:489 _disallow_when_autograph_enabled
        raise errors.OperatorNotAllowedInGraphError(

    OperatorNotAllowedInGraphError: iterating over `tf.Tensor` is not allowed: AutoGraph did convert this function. This 